In [1]:
from utils.data_preprocessing import *
import os
import numpy as np

In [2]:
base_path = os.path.join('..', 'data', 'dlis_preprocessed')
csv_data = load_csv_files(base_path)

In [3]:
gr_data = create_df_subset(csv_data, 'GR')

In [4]:
for well, w_dict in gr_data.items():
    for logical_file, lf_dict in w_dict.items():
        for frame, df in lf_dict.items():
            try:
                # Remove lines with NaN
                df.dropna(inplace=True)

                max_tdep = max(max_tdep, df['TDEP'].max())
            except:
                pass

C:\Users\rafae\AppData\Local\Temp\ipykernel_5936\2261918962.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
C:\Users\rafae\AppData\Local\Temp\ipykernel_5936\2261918962.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
C:\Users\rafae\AppData\Local\Temp\ipykernel_5936\2261918962.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
C:\Users\rafae\AppData\Local\Te

In [5]:
spliced_data = {}
for well, w_dict in gr_data.items():
    well_data = []

    for logical_file, lf_dict in w_dict.items():
        for frame, df in lf_dict.items():
            well_data.append(df)

    # Concat all the dfs on the vertical axis
    df_concat = pd.concat(well_data)

    # Group by TDEP and keep the max value of GR
    df_merged = df_concat.groupby('TDEP', as_index=False).GR.max()

    spliced_data[well] = df_merged

In [6]:
spliced_dfs_to_csv(spliced_data, '../data/dlis_spliced/GR')